## Badcase Analysis

In [1]:
import time
import numpy as np
import pandas as pd

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from xgboost.sklearn import XGBClassifier

from tianchi.user_location_prediction.pysrc.feature import *
from tianchi.user_location_prediction.pysrc.load import *

/home/zhouzr/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
sample = load_sample()
shop = load_shop()
sample_wifi = load_sample_wifi()

/home/zhouzr/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [8]:
with open('/home/zhouzr/project/zhihu_public_code/tianchi/user_location_prediction/record/mall_score') as f:
    record = f.readlines()
record = np.array([float(i.rstrip('\n')) for i in record])

In [12]:
analysis = pd.DataFrame({'mall_id':sample.mall_id.unique(), 'acc_cv0':record}, columns=['mall_id','acc_cv0'])

In [29]:
mall_classcnt = sample.groupby('mall_id')['shop_id'].nunique()
mall_samplecnt = sample.groupby('mall_id')['shop_id'].count()

mall_wificnt = {}
for i,j in sample.groupby('mall_id')['sample_id']:
    mall_wificnt[i] = sample_wifi[sample_wifi.sample_id.isin(j.values)]['wifi_id'].nunique()

In [35]:
mall_wificnt = pd.Series(mall_wificnt)

In [42]:
mall_info=pd.DataFrame({'class_num':mall_classcnt, 'sample_num':mall_samplecnt, 'wifi_num':mall_wificnt}).reset_index()

In [46]:
mall_info = mall_info.merge(analysis, on='mall_id')

In [48]:
mall_info.corr()

,class_num,sample_num,wifi_num,acc_cv0
class_num,1.000000,0.455157,0.549861,-0.252622
sample_num,0.455157,1.000000,0.764483,-0.452816
wifi_num,0.549861,0.764483,1.000000,-0.215019
acc_cv0,-0.252622,-0.452816,-0.215019,1.000000


In [49]:
mall_info[mall_info.acc_cv0<0.8]

,mall_id,class_num,sample_num,wifi_num,acc_cv0
12,m_1920,69,15301,6640,0.787988
45,m_4079,91,20619,5811,0.770260
49,m_4187,89,9520,2754,0.755777
52,m_4422,94,20741,6491,0.752278
72,m_6167,124,5358,3914,0.799179
81,m_7168,99,26355,8739,0.728932
85,m_7800,89,19732,4489,0.700689


'm_7800'